# Options Screener

This notebook is meant to analyze a list of tickers and output which tickers have the highest value call options with the highest probability of increasing in value in the short-term.  

Manual steps procedure:  
- Update options_list2.csv with the latest OI pct changes from Optionistics.com using the options_list_compare.xlsx file  
- Get the latest finviz file from the local directory  

This screener looks at the following attributes:  
- Whether an option is 'cheap' i.e. has low realized volatility compared to its historical realized volatility, the lowest 15% are given a point  
- Implied volatility premium or discount, the top 15% are given a point  
- Open interest % change from optionistics, these are given a point  
- Highest float short percent, the top 15% are given a point

In [139]:
# %matplotlib inline

import math
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

In [140]:
import pandas as pd

tickers_df = pd.read_csv('../0_inputs/asset_screeners/options_list1.csv')
optionistics_data = pd.read_csv('../0_inputs/asset_screeners/options_list2.csv')
# optionistics_data.head(3)

ticker_dictionary = {
    'VIX':'^VIX'
}

ticker_list_df = pd.merge(tickers_df[['ticker']],optionistics_data[['ticker']],on='ticker',how='outer')

ticker_list = ticker_list_df['ticker'].to_list()

In [141]:
def realized_vol(price_data, window=30):

#     print(price_data.tail(5))
    log_return = (price_data["Close"] / price_data["Close"].shift(1)).apply(np.log)

    return log_return.rolling(window=window, center=False).std() * math.sqrt(252)

In [142]:
windows = [30, 60, 90, 120]
quantiles = [0.25, 0.75]

min_ = []
max_ = []
median = []
top_q = []
bottom_q = []
realized = []

In [143]:
completed_tkr_list = []
realized_vol_list = []
avg_pct_list = []
pct30_list = []
pct60_list = []
pct90_list = []
pct120_list = []
counter = 1

cur_price_dict = {}

# ticker_list = ['AAPL','TSLA','MSFT'] # test list

for ticker in ticker_list:

    print(f"Processing {ticker}, {counter} of {len(ticker_list)}...")
    # data = yf.download("SPY AAPL", start="2017-01-01", end="2017-04-30")

    # start="2020-01-01", end="2020-12-31"

    # start_date = '2020-01-01'
    # end_date = '2020-12-31'
    
    if ticker in ticker_dictionary:
        tkr = ticker_dictionary[ticker]
    else: tkr = ticker

    data = yf.download(  # or pdr.get_data_yahoo(...
            # tickers list or string as well
            tickers = tkr,

            # use "period" instead of start/end
            # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
            # (optional, default is '1mo')
            period = "1y",

    #         start = start_date,
    #         end = end_date,

            # fetch data by interval (including intraday if period < 60 days)
            # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            # (optional, default is '1d')
            interval = "1d",

            # group by ticker (to access via data['SPY'])
            # (optional, default is 'column')
            group_by = 'ticker',

            # adjust all OHLC automatically
            # (optional, default is False)
            auto_adjust = True,

            # download pre/post regular market hours data
            # (optional, default is False)
            prepost = True,

            # use threads for mass downloading? (True/False/Integer)
            # (optional, default is True)
            threads = True,

            # proxy URL scheme use use when downloading?
            # (optional, default is None)
            proxy = None
        )

    # data.to_csv('BTC_1hr_prices.csv')
    # data.head(3)
#     print(data.tail(5))

    cur_price_dict[ticker] = data['Close'][-1]

    min_ = []
    max_ = []
    median = []
    top_q = []
    bottom_q = []
    realized = []

    for window in windows:

        # get a dataframe with realized volatility
        estimator = realized_vol(window=window, price_data=data)
#         print(estimator)

        # append the summary stats to a list
        min_.append(estimator.min())
        max_.append(estimator.max())
        median.append(estimator.median())
        top_q.append(estimator.quantile(quantiles[1]))
        bottom_q.append(estimator.quantile(quantiles[0]))
        realized.append(estimator[-1])
#         print(min_)

    pct_list = []

    pct30 = (realized[0] - bottom_q[0]) / (top_q[0] - bottom_q[0]) * 100
    pct_list.append(pct30)
    pct60 = (realized[1] - bottom_q[1]) / (top_q[1] - bottom_q[1]) * 100
    pct_list.append(pct60)
    pct90 = (realized[2] - bottom_q[2]) / (top_q[2] - bottom_q[2]) * 100
    pct_list.append(pct90)
    pct120 = (realized[3] - bottom_q[3]) / (top_q[3] - bottom_q[3]) * 100
    pct_list.append(pct120)

#     print(pct30)
#     print(pct60)
#     print(pct90)
#     print(pct120)
#     print(pct_list)

    avg_pct = sum(pct_list)/len(pct_list)
#     print(avg_pct)
    
    completed_tkr_list.append(ticker)
    realized_vol_list.append(round(realized[1] * 100,2)) # this is realized vol for 60 day window
    avg_pct_list.append(round(avg_pct,2))
    pct30_list.append(round(pct30,2))
    pct60_list.append(round(pct60,2))
    pct90_list.append(round(pct90,2))
    pct120_list.append(round(pct120,2))
    
    counter += 1

Processing SPY, 1 of 293...
[*********************100%***********************]  1 of 1 completed
Processing QQQ, 2 of 293...
[*********************100%***********************]  1 of 1 completed
Processing SPXU, 3 of 293...
[*********************100%***********************]  1 of 1 completed
Processing AAPL, 4 of 293...
[*********************100%***********************]  1 of 1 completed
Processing AMZN, 5 of 293...
[*********************100%***********************]  1 of 1 completed
Processing TSLA, 6 of 293...
[*********************100%***********************]  1 of 1 completed
Processing AMD, 7 of 293...
[*********************100%***********************]  1 of 1 completed
Processing VIX, 8 of 293...
[*********************100%***********************]  1 of 1 completed
Processing IWM, 9 of 293...
[*********************100%***********************]  1 of 1 completed
Processing HYG, 10 of 293...
[*********************100%***********************]  1 of 1 completed
Processing META, 11 of 29

[*********************100%***********************]  1 of 1 completed
Processing NLOK, 167 of 293...
[*********************100%***********************]  1 of 1 completed
Processing DAL, 168 of 293...
[*********************100%***********************]  1 of 1 completed
Processing GLBS, 169 of 293...
[*********************100%***********************]  1 of 1 completed
Processing CLF, 170 of 293...
[*********************100%***********************]  1 of 1 completed
Processing ADVM, 171 of 293...
[*********************100%***********************]  1 of 1 completed
Processing AEM, 172 of 293...
[*********************100%***********************]  1 of 1 completed
Processing TEF, 173 of 293...
[*********************100%***********************]  1 of 1 completed
Processing IPOD, 174 of 293...
[*********************100%***********************]  1 of 1 completed
Processing GFI, 175 of 293...
[*********************100%***********************]  1 of 1 completed
Processing AQMS, 176 of 293...
[****

In [144]:
import pandas as pd

pct_dictionary = {
    'ticker':completed_tkr_list,
    'realized_vol':realized_vol_list,
    'avg_pct':avg_pct_list,
    'pct30':pct30_list,
    'pct60':pct60_list,
    'pct90':pct90_list,
    'pct120':pct120_list
}

cheap_df = pd.DataFrame(pct_dictionary)
cheap_df = cheap_df.sort_values(['avg_pct'])
cheap_df.to_csv('../0_data_dump/optionistics/realized_vol.csv')
cheap_df

,ticker,realized_vol,avg_pct,pct30,pct60,pct90,pct120
47,IMPP,159.91,-338.65,-34.76,-5.06,-191.99,-1122.77
173,IPOD,2.35,-246.54,-272.65,-373.59,-135.51,-204.39
188,LUMN,39.67,-196.65,-12.78,16.47,1.52,-791.81
29,UVXY,87.49,-195.50,-80.96,-446.54,-64.05,-190.43
224,GILT,33.04,-142.33,-77.45,-7.98,-40.24,-443.64
...,...,...,...,...,...,...,...
254,GBT,108.21,493.74,386.02,358.17,582.08,648.69
245,KERN,183.16,532.59,615.24,497.67,414.60,602.83
102,TEVA,69.55,651.04,437.04,1003.68,608.48,554.94
87,HGEN,345.94,822.11,971.40,1004.34,609.80,702.89


In [145]:
import glob
import os

# Get the latest file from the directory
list_of_files = glob.glob('../0_inputs/asset_screeners/Finviz_Data/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)

../0_inputs/asset_screeners/Finviz_Data\finviz (42).csv


In [146]:
import pandas as pd
import numpy as np

col_names = {
    'Ticker':'ticker',
    'Float Short':'float_short'
}

# Bring finviz.csv file into dataframe
finviz_df = pd.read_csv(latest_file)
finviz_df = finviz_df.rename(columns=col_names)
finviz_df = finviz_df[['ticker','float_short']]
finviz_df['float_short'] = finviz_df['float_short'].str.slice(0,-1) # remove the % character from the end of the string
finviz_df['float_short'] = finviz_df['float_short'].astype(str).astype(float) # convert to float for proper sorting
finviz_df = finviz_df.sort_values(['float_short'],ascending=False)

# Add Relative Volume Logic here
finviz_df['float_short_pt'] = 0
finviz_df['float_short_pt'] = np.select(condlist=[(finviz_df['float_short']>finviz_df['float_short'].quantile(0.85))],
                           choicelist=[1],
                           default=0)

# finviz_df = finviz_df[finviz_df['float_short'] > finviz_df['float_short'].quantile(.85)]
# finviz_df2 = finviz_df[finviz_df['float_short_pt']==1]
finviz_df.head(3)

,ticker,float_short,float_short_pt
6446,REV,86.52,1
3662,HRTX,56.39,1
730,BBBY,41.73,1


In [147]:
# Optional for if internet is running slow
# cheap_df = pd.read_csv('../0_data_dump/optionistics/realized_vol_output.csv')
# cheap_df = cheap_df.drop('Unnamed: 0', axis=1)

# Add Relative Volume Logic here
cheap_df['cheap_pt'] = 0
cheap_df['cheap_pt'] = np.select(condlist=[(cheap_df['avg_pct'] < 0)],
                           choicelist=[1],
                           default=0)

cheap_df.head(3)

,ticker,realized_vol,avg_pct,pct30,pct60,pct90,pct120,cheap_pt
47,IMPP,159.91,-338.65,-34.76,-5.06,-191.99,-1122.77,1
173,IPOD,2.35,-246.54,-272.65,-373.59,-135.51,-204.39,1
188,LUMN,39.67,-196.65,-12.78,16.47,1.52,-791.81,1


Implied volatility premium or discount calculations here.

In [148]:
import yfinance as yf
import datetime
import sys

today_date = datetime.datetime.now()
target_date = today_date + datetime.timedelta(days=60)

# ticker_list = ['AAPL','TSLA','MSFT'] # test list

ticker_list2 = []
expiry_list = []
strike_list = []
iv_list = []
counter = 1

for ticker in ticker_list:
    
    try:
        print(f'\nProcessing {ticker}, {counter} of {len(ticker_list)}...')
        
        # options_data = yf.Ticker("MSFT") # test input
        options_data = yf.Ticker(ticker)
        option_dates_list = options_data.options

        closest_date = today_date

        for x in option_dates_list:
            x = datetime.datetime.strptime(x, '%Y-%m-%d')
        #     print(x,',',(x-target_date).days)
            if (abs((x - target_date).days)) < abs(((closest_date - target_date).days)):
                closest_date = x

        # print(closest_date)
        # days_to_expiration = (closest_date - today_date).days
        # print(days_to_expiration)
        closest_date = closest_date.strftime("%Y-%m-%d")
        print(f'Closest expiry is: {closest_date}')

        opt = options_data.option_chain(closest_date)
        strike_price_np_array = opt[0]['strike'].to_numpy()

        if ticker in cur_price_dict:
            cur_price = cur_price_dict[ticker]
        else:
            data = yf.download(
                tickers = tkr,
                period = "5d",
                interval = "1d",
                group_by = 'ticker',
                auto_adjust = True,
                prepost = True,
                threads = True,
                proxy = None
            )
            cur_price = data['Close'][-1]
        
#         cur_price = 280 # test input
        closest_strike = 0

        strike_ctr = 0
        
        for x in strike_price_np_array:
            if abs(cur_price - x) < abs(cur_price - closest_strike):
                closest_strike = x
                strike_index = strike_ctr
            strike_ctr += 1

        print(f'Closest strike price is: {closest_strike}')

        strike_df = opt[0][opt[0]['strike']==closest_strike]
        implied_vol = round(strike_df['impliedVolatility'][strike_index] * 100,2)
        print(f'Implied volatility is: {implied_vol}')

        ticker_list2.append(ticker)
        expiry_list.append(closest_date)
        strike_list.append(closest_strike)
        iv_list.append(implied_vol)
        counter += 1
    
    except Exception as e:
        print(f'{ticker}, {counter} not found.')
        print(f'Exception is {e}')
        print(f'Error is: {sys.exc_info()}')
        counter += 1


Processing SPY, 1 of 293...
Closest expiry is: 2022-09-30
Closest strike price is: 413.0
Implied volatility is: 19.98

Processing QQQ, 2 of 293...
Closest expiry is: 2022-09-30
Closest strike price is: 322.0
Implied volatility is: 26.6

Processing SPXU, 3 of 293...
Closest expiry is: 2022-09-16
Closest strike price is: 15.0
Implied volatility is: 58.79

Processing AAPL, 4 of 293...
Closest expiry is: 2022-10-21
Closest strike price is: 165.0
Implied volatility is: 29.36

Processing AMZN, 5 of 293...
Closest expiry is: 2022-10-21
Closest strike price is: 140.0
Implied volatility is: 37.19

Processing TSLA, 6 of 293...
Closest expiry is: 2022-10-21
Closest strike price is: 860.0
Implied volatility is: 59.06

Processing AMD, 7 of 293...
Closest expiry is: 2022-10-21
Closest strike price is: 100.0
Implied volatility is: 50.06

Processing VIX, 8 of 293...
Closest expiry is: 2022-08-06
VIX, 8 not found.
Exception is Expiration `2022-08-06` cannot be found. Available expiration are: []
Error

Closest expiry is: 2022-09-16
Closest strike price is: 1.0
Implied volatility is: 164.06

Processing BIOR, 68 of 293...
Closest expiry is: 2022-10-21
Closest strike price is: 1.0
Implied volatility is: 120.31

Processing RIG, 69 of 293...
Closest expiry is: 2022-09-16
Closest strike price is: 3.5
Implied volatility is: 91.8

Processing PFE, 70 of 293...
Closest expiry is: 2022-10-21
Closest strike price is: 50.0
Implied volatility is: 28.22

Processing MMAT, 71 of 293...
Closest expiry is: 2022-10-21
Closest strike price is: 1.0
Implied volatility is: 102.34

Processing C, 72 of 293...
Closest expiry is: 2022-10-21
Closest strike price is: 52.5
Implied volatility is: 33.37

Processing MULN, 73 of 293...
Closest expiry is: 2022-10-21
Closest strike price is: 1.0
Implied volatility is: 178.13

Processing CGC, 74 of 293...
Closest expiry is: 2022-10-21
Closest strike price is: 2.5
Implied volatility is: 97.66

Processing AGNC, 75 of 293...
Closest expiry is: 2022-09-16
Closest strike pric

Closest expiry is: 2022-10-21
Closest strike price is: 10.0
Implied volatility is: 56.74

Processing WISH, 130 of 293...
Closest expiry is: 2022-10-21
Closest strike price is: 2.0
Implied volatility is: 97.66

Processing GM, 131 of 293...
Closest expiry is: 2022-09-16
Closest strike price is: 36.0
Implied volatility is: 42.16

Processing ABEO, 132 of 293...
Closest expiry is: 2022-09-16
Closest strike price is: 5.0
Implied volatility is: 60.55

Processing ASHR, 133 of 293...
Closest expiry is: 2022-10-21
Closest strike price is: 31.0
Implied volatility is: 23.98

Processing PARA, 134 of 293...
Closest expiry is: 2022-10-21
Closest strike price is: 25.0
Implied volatility is: 47.61

Processing HTZ, 135 of 293...
Closest expiry is: 2022-09-16
Closest strike price is: 22.5
Implied volatility is: 52.15

Processing JBLU, 136 of 293...
Closest expiry is: 2022-09-16
Closest strike price is: 9.0
Implied volatility is: 50.98

Processing BCS, 137 of 293...
Closest expiry is: 2022-09-16
Closest s

Closest expiry is: 2022-09-16
Closest strike price is: 10.0
Implied volatility is: 38.97

Processing SKLZ, 194 of 293...
Closest expiry is: 2022-09-16
Closest strike price is: 1.5
Implied volatility is: 108.59

Processing BB, 195 of 293...
Closest expiry is: 2022-09-16
Closest strike price is: 7.0
Implied volatility is: 59.57

Processing CIM, 196 of 293...
Closest expiry is: 2022-09-16
Closest strike price is: 9.0
Implied volatility is: 48.05

Processing M, 197 of 293...
Closest expiry is: 2022-09-16
Closest strike price is: 18.0
Implied volatility is: 68.56

Processing CPG, 198 of 293...
Closest expiry is: 2022-10-21
Closest strike price is: 7.5
Implied volatility is: 71.48

Processing BRFS, 199 of 293...
Closest expiry is: 2022-10-21
Closest strike price is: 2.5
Implied volatility is: 66.41

Processing LFLY, 200 of 293...
Closest expiry is: 2022-10-21
Closest strike price is: 2.5
Implied volatility is: 94.92

Processing SWN, 201 of 293...
Closest expiry is: 2022-09-16
Closest strike 

Closest expiry is: 2022-09-16
Closest strike price is: 50.0
Implied volatility is: 84.52

Processing ACWI, 256 of 293...
Closest expiry is: 2022-10-21
Closest strike price is: 90.0
Implied volatility is: 21.29

Processing NEOG, 257 of 293...
Closest expiry is: 2022-10-21
Closest strike price is: 22.5
Implied volatility is: 65.38

Processing KRTX, 258 of 293...
Closest expiry is: 2022-09-16
Closest strike price is: 140.0
Implied volatility is: 131.06

Processing CSTM, 259 of 293...
Closest expiry is: 2022-09-16
Closest strike price is: 14.0
Implied volatility is: 51.95

Processing STEM, 260 of 293...
Closest expiry is: 2022-10-21
Closest strike price is: 15.0
Implied volatility is: 85.01

Processing AES, 261 of 293...
Closest expiry is: 2022-09-16
Closest strike price is: 24.0
Implied volatility is: 39.94

Processing DASH, 262 of 293...
Closest expiry is: 2022-10-21
Closest strike price is: 80.0
Implied volatility is: 68.85

Processing TWLO, 263 of 293...
Closest expiry is: 2022-10-21
C

In [149]:
print('Completed.')

Completed.


Create a dataframe for the implied volatility data.

In [150]:
iv_dictionary = {
    'ticker':ticker_list2,
    'expiry':expiry_list,
    'strike':strike_list,
    'implied_vol':iv_list
}

iv_df = pd.DataFrame(iv_dictionary)
iv_df["implied_vol"] = iv_df["implied_vol"].astype(str).astype(float)
iv_df = iv_df.sort_values(['implied_vol'],ascending=False)
iv_df.to_csv('../0_data_dump/optionistics/implied_vols.csv')
iv_df.head(3)

,ticker,expiry,strike,implied_vol
129,AMRN,2022-10-21,1.0,496.88
242,PACB,2022-10-21,6.0,230.66
199,UXIN,2022-09-16,1.0,196.88


In [151]:
print('Completed.')

Completed.


In [152]:
final_df = pd.merge(cheap_df,finviz_df,on='ticker',how='outer')
final_df = pd.merge(final_df,optionistics_data,on='ticker',how='outer')
final_df = pd.merge(final_df,iv_df,on='ticker',how='outer')
final_df['iv_premium'] = final_df['implied_vol'] - final_df['realized_vol']

final_df['iv_prem_pt'] = 0
final_df['iv_prem_pt'] = np.select(condlist=[(final_df['iv_premium']>final_df['iv_premium'].quantile(0.85))],
                           choicelist=[1],
                           default=0)

final_df['oi_pct_chg_pts'] = final_df['oi_pct_chg_pts'].fillna(0)
final_df['cheap_pt'] = final_df['cheap_pt'].fillna(0)
final_df['total_pts'] = final_df['cheap_pt'] + final_df['float_short_pt'] + final_df['oi_pct_chg_pts'] + final_df['iv_prem_pt']
final_df = final_df[['ticker','expiry','strike','total_pts','iv_prem_pt','cheap_pt','oi_pct_chg_pts','float_short_pt','iv_premium','avg_pct','float_short','oi_pct_chg','p_c_ratio']]
final_df = final_df.sort_values(by = ['total_pts','float_short'],ascending=[False,False])
final_df = final_df[final_df['total_pts'] > 0]
final_df.to_csv('../0_data_dump/asset_screener/options_output.csv')
final_df.head(20)

,ticker,expiry,strike,total_pts,iv_prem_pt,cheap_pt,oi_pct_chg_pts,float_short_pt,iv_premium,avg_pct,float_short,oi_pct_chg,p_c_ratio
26,BKKT,2022-09-16,3.00,3.0,0,1.0,1.0,1.0,-4.41,-17.16,33.20,8.06,0.47
209,PACB,2022-10-21,6.00,3.0,1,0.0,1.0,1.0,118.19,126.31,17.59,15.63,0.71
9,DOCS,2022-09-16,37.50,3.0,0,1.0,1.0,1.0,-9.79,-70.96,16.90,13.31,0.98
15,PLBY,2022-10-21,7.50,3.0,1,1.0,0.0,1.0,15.17,-42.93,15.08,NaN,NaN
25,BIRD,2022-09-16,5.00,3.0,1,1.0,0.0,1.0,12.06,-17.77,13.14,NaN,NaN
23,BIOR,2022-10-21,1.00,3.0,1,1.0,0.0,1.0,34.84,-20.48,10.85,NaN,NaN
20,MNMD,2022-09-16,0.50,3.0,1,1.0,0.0,1.0,63.12,-24.46,8.67,NaN,NaN
225,BYND,2022-09-16,40.00,2.0,0,0.0,1.0,1.0,-38.89,133.73,37.13,6.97,1.50
230,CVNA,2022-10-21,45.00,2.0,0,0.0,1.0,1.0,-51.03,137.15,37.01,14.65,0.77
149,BIG,2022-10-21,22.50,2.0,0,0.0,1.0,1.0,-9.97,91.78,35.19,7.53,0.78


In [153]:
print(f'Completed on {datetime.datetime.now()}.')

Completed on 2022-08-06 23:03:36.037038.
